# JSX / React

JSX je extension, který do jazyka přinesl React. JSX umožňuje psát částo kódu jako by se jednalo o HTML kód. Mezi tagy HTML je možné do složených závorek vložit Javascriptový kód.
JSX vyžaduje, aby tagy byly uzavřeny. Javascript překládá JSX do objektu, jehož prvkem může být funkce.

## Pomocné prvky

Pro řádné fungování následující části je nezbytné, aby byly instalovány knihovny react a react-dom. Knihovny nainstalujete spuštěním následujících příkazů na příkazovém řádku v příslušném kontejneru. Instalaci lze provést i prostřednictvím terminálu v jupyteru.
```
npm install react
npm install react-dom
```

> Pozor, nespouštět s přepínačem `-g`, nebylo by funkční.

Následující funkce `DisplayReact` usnadní demonstraci některých principů v tomto notebooku. Očekávaný parametrem této funkce je komponenta.

In [2]:
import ReactDOMServer from 'react-dom/server';
import * as tslab from 'tslab';

function DisplayReact(element) {
    const html = ReactDOMServer.renderToString(element);
    tslab.display.html(html);
}

Demonstrace použití funkce `DisplayReact`.

In [4]:
import React from 'react';

const Hello = (props) => <div><hr />Hello {props.name}<hr /></div>;
DisplayReact(<Hello name='World' />);

Hello <!-- -->World

## Rychlý rozbor výstupů funkcí

Komponenta v reactjs je definována jako funkce. Je možné definovat komponentu jako třídu. Je silně doporučováno definovat komponenty jako funkce. Nutnost definovat vnitřní stav komponenty (případně další skutečnosti) je možno realizovat pomocí tzv hooks.

Je vhodné vědět, co vlastně funkce vrací. Níže je ukázáno, že se jedná o speciální datovou strukturu.

In [5]:
import React from 'react';

const NameElement = (props) =>
    <div>Hello {props.name ? props.name : ''}</div>;

console.log(NameElement({name: 'Hrbolek'}));

{
  '$$typeof': Symbol(react.element),
  type: 'div',
  key: null,
  ref: null,
  props: { children: [ 'Hello ', 'Hrbolek' ] },
  _owner: null,
  _store: {}
}


V případě, kdy použijeme výraz, dostaneme odlišný výsledek. Tento výsledek lze chápat jako lazy value.

In [6]:
import React from 'react';

const htmlElement = <NameElement name='Hrbolek'/>;
console.log(htmlElement);

{
  '$$typeof': Symbol(react.element),
  type: [Function: NameElement],
  key: null,
  ref: null,
  props: { name: 'Hrbolek' },
  _owner: null,
  _store: {}
}


Pro získání výstupu je nutné "převést lazy value na value". Toto lze udělat voláním funkce, která lazy value představuje.

In [12]:
import React from 'react';

const htmlElement = <NameElement name='Hrbolek'/>;
const result = htmlElement.type(htmlElement.props)
console.log(result);

{
  '$$typeof': Symbol(react.element),
  type: 'div',
  key: null,
  ref: null,
  props: { children: [ 'Hello ', 'Hrbolek' ] },
  _owner: null,
  _store: {}
}


Dříve definovaná funkce `DisplayReact` si dokáže poradit s oběma typy výstupů. V případě, kdy je předaná lazy value, je nejdříve provedeno vyhodnocení. Funkce `DisplayReact` toto zabezpečuje voláním funkce `ReactDOMServer.renderToString`. V případě, kdy se provádí vizualizace na klientovi (prohlížeč), používá se jiná funkce, která zabezpečuje synchronizaci s aktuálně zobrazeným DOM (document object model).

In [14]:
DisplayReact(result)
DisplayReact(htmlElement)

Hello <!-- -->Hrbolek

Hello <!-- -->Hrbolek

## Parametry pro komponenty

Komponenta má jediný parametr a ten bývá formálně pojmenováván jako `props`. V případě, kdy je komponenta použita (ekvivalent html), je struktura `props` odvozena od atributů (`id={2}`).

In [15]:
import React from 'react';

const Page = (props) => <div>Page {props.id}</div>;
DisplayReact(<Page id={2} />);

Page <!-- -->2

Toto platí i v případě, kdy je atributů více.

In [16]:
import React from 'react';

const UserSmall = (props) => <div>{props.name} {props.surname}</div>;
DisplayReact(<UserSmall name='John' surname='Winkl' />);

John<!-- --> <!-- -->Winkl

Speciálním případem je skutečnost, kdy komponenta obaluje další komponenty. Tyto vnořené komponenty jsou uloženy v `props.children`. Tato skutečnost umožňuje provádět kompozici komponent a řešit tím celou řadu problémů.

In [18]:
import React from 'react';

const Container = (props) => <div><hr />{props.children}<hr /></div>;
DisplayReact(<Container><Page id={2} /></Container>);

Page <!-- -->2

Prostřednictvím `props` je možné předat funkci, případně komplecní datové struktury či cokoliv, co může být v jazyku Javascript uloženo do proměnné. Funkce je demonstrována níže.

In [20]:
import React from 'react';

const addX = (value) => value + 'X';

const Visualiser = (props) => <div>{props.func(props.data)}</div>;
DisplayReact(<Visualiser data={'A'} func={addX}/>);

AX

Funkce představující komponentu nemusí být jen "arrow function". Je možné syntakticky použít všechny funkce.

In [23]:
import React from 'react';

function Visualiser(props) {
    return <div>{props.data}</div>;
}

DisplayReact(<Visualiser data={'A'} />);

A

## Strukturalizace webové stránky v ReactJS

### Layout

Velmi často má webová aplikace společný layout. Typicky je součástí tohoto layoutu menu.

In [24]:
import React from 'react';

function Layout(props) {
    return <div><hr />Menu<hr />{props.children} <hr/></div>;
}

DisplayReact(<Layout />);

Menu

### Container

Dalším prvkem je kontejner, do kterého se vkládá vlastní obsah stránky.

In [31]:
import React from 'react';

function Container(props) {
    return <div style={{'backgroundColor': '#c6ecc6'}}>{props.children} </div>;
}

DisplayReact(<Layout><Container>Hello World</Container></Layout>);

Menu Hello World<!-- -->

### Elementární datové prvky

Uživatel chce, aby na stránce, která mu poskytuje informace, byly jednotlivé prvky, o kterých je na stránce zmínka, provázány na jiné stránky, které k těmto prvků poskytují komplexní popis (informace o skupině - member - informace o uživateli).

Takové prvky je vhodné vytvářet ve formě komponent a jejich obsah upravovat v souladu s potřebou směrovat v uživatelském prostředí navigaci na "cílovou informaci". V implementaci níže by bylo vhodné do prvku `<a />`, doplnit atribut href, který by zabezpečil požadované navedení.

In [45]:
import React from 'react';

function UserSmall(props) {
    return <a>{props.name} {props.surname}</a>;
}

DisplayReact(
    (
    <Layout>
        <Container>
            <UserSmall name={'John'} surname={'Winkl'} />
        </Container>
    </Layout>
));

Menu John<!-- --> <!-- -->Winkl

Níže uvedená komponenta slouží ke zobrazení seznamu (user list). Všimněte si použití metody `map` u datového typu `Array` pro vytvoření vizuálních prvků.

In [47]:
import React from 'react';

function UserList(props) {
    return (<ul>{props.members.map(
               (user, index) => (
                    <li key={index}>
                        <UserSmall key={1} {...user} />
                        <br key={'b'}/>
                    </li>)
               )}</ul>)
}

const members = [
    {'name': 'John', 'surname': 'Winkl'},
    {'name': 'Peter', 'surname': 'Wild'},
    {'name': 'Will', 'surname': 'Sword'}
]

DisplayReact(
    (
    <Layout>
        <Container>
            <UserList members={members} />
        </Container>
    </Layout>
));

Menu John<!-- --> <!-- -->Winkl Peter<!-- --> <!-- -->Wild Will<!-- --> <!-- -->Sword

Složitou komponentu (`DepartmentLarge`) je vhodné rozložit do menších komponent (`DepartmentMembers`, `DepartmentTitle`). Tímto rozkladem jednak zvyšujeme čitelnost výsledného kódu a jednak zabezpečujeme vyšší flexibilitu při vývoji uživatelského rozhraní.

In [48]:
import React from 'react';

const DeparmentTitle = (props) => (
    <>
        <b>Name</b><br />
        {props.name}<br />
    </>
)

const DeparmentMembers = (props) => (
    <>
        <b>Member List</b><br />
        <UserList {...props} /><br />
    </>
)

function DeparmentLarge(props) {
    return (
        <>
            <DeparmentTitle {...props} />
            <DeparmentMembers {...props} />
        </>);
}

DisplayReact(
    (
    <Layout>
        <Container>
            <DeparmentLarge name={'KIT'} members={members} />
        </Container>
    </Layout>
));

Menu Name KIT Member List John<!-- --> <!-- -->Winkl Peter<!-- --> <!-- -->Wild Will<!-- --> <!-- -->Sword

### Stories a final

Existují nástroje, s jejichž pomocí lze vytvářet katalogy komponent (`Story book`). Zajistit si komponentu (`DepartmentStoryBook`), která může do tohot katalogu být zařazena a která současně poskytuje implicitní datové struktury pro vizualizaci vnořených komponent, lze považovat za dobrou praxi.

Komunikaci se serverem je nutné zajistit (v nejjedodušší variantě) pomocí `useState` a `useEffect`, což jsou tzv. hooks, které slouží k provedení asynchronního kódu a uložení výsledku (result of `fetch`). Tato komunikace je zapouzdřena v komponentě `DepartmentFecthing`. Pozor, vlastní `fetch` (obsah `useEffect`) zde není implementován.

Komponenta `DepartmentPage` poslouží v případě, že jsou používány technologie `router` pro single page application, či podobné.

In [44]:
import React from 'react';
import { useState, useEffect } from "react";

const DeparmentStoryBook = (props) => {
    const extraProps = {
        'name': 'KIT',
        'members': [
            {'name': 'John', 'surname': 'Winkl'},
            {'name': 'Peter', 'surname': 'Wild'},
            {'name': 'Will', 'surname': 'Sword'}        
        ]
    }
    return <DeparmentLarge {...extraProps} {...props} />
}
    
const DepartmentFetching = (props) => {
    const [state, setState] = useState(props);
    useEffect(
        () => {}, [props.id]);
    return <DeparmentStoryBook {...state} />
}
    
const DepartmentPage = (props) => {
    return <DepartmentFetching id={3} />
}
    
DisplayReact(
    (
    <Layout>
        <Container>
            <DepartmentPage />
        </Container>
    </Layout>
));

Menu Name KIT Member List John<!-- --> <!-- -->Winkl Peter<!-- --> <!-- -->Wild Will<!-- --> <!-- -->Sword

## Cvičení

Inspirujte se následující komponentou a upravte předcházející s cílem vizuálně oddělit jednotlivé prvky stránky zobrazující informace o skupině.

In [58]:
import React from 'react';

const ColorBar = (props) => <div style={{'backgroundColor': props.color}}>{props.children}<br/></div>

DisplayReact(
    (
    <>
        <ColorBar color='#ff8c1a'/>
        <ColorBar color='#ffcc99'>Hello</ColorBar>
        <ColorBar color='#ff8c1a'/>
    </>)
)

Hello